In [2]:
from collections import defaultdict
import logging

import pandas as pd
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import json
import itertools

from db.manager import DBManager
from pathlib import Path

INFO:db.instance:Connected to the database PerSite_DB


In [3]:
BASE_DIR = "../"

In [8]:
class EM:
    json_structure = "JsonStructureCorrectness"
    true_positive = "TruePositive"
    false_positive = "FalsePositive"
    false_negative = "FalseNegative"

def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        result = json.loads(f.read())
    
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in result]
    return result

def run_query_and_get_report(input, scenario, instruction_set):
    input_report = {}
    input_report["Input"] = input
    input_report["Scenario"] = scenario
    input_report["Result"] = []
    for instruction in instruction_set:
        i_type = instruction["type"]
        if i_type == "q":
            # query
            args = instruction["args"]
            result_var_name = instruction["result_name"]
            # print(f"Query: {args}, {result_var_name}")
            result_df = DBManager.structured_query(args, get_rowids=True)
            # print(f"Result:\n{result_df}")
            try:
                if "timestamp" in result_df.columns:
                    try:
                        result_df["timestamp"] = result_df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
                    except Exception as e:
                        print(args)
                        print(result_df["timestamp"])
                result = result_df.to_dict(orient="index")
                cols = list(result_df.columns)
                result = [[row[col] for col in cols] for row in result.values()]
                
                input_report["Result"].append({
                    "type": "q",
                    "args": args,
                    # "result_name": result_var_name,
                    "result_shape": result_df.shape,
                    "result_columns": cols,
                    "result_indices": list(result_df["id"]),
                    # "result": result
                })
            except Exception as e:
                logger.error(f"Error inside: {e}")
                logger.error(f"Invoked with Query: {args}, {result_var_name}")
        
    return input_report

def build_query_groundtruth():
    ds_ts = []
    dt_tr = []
    base_dataset_dir = Path(f"{BASE_DIR}/finetuning/dataset/v5-250228-multimetadata")
    for directory in base_dataset_dir.iterdir():
        if directory.is_dir() and "scenario" in directory.name:
            ts = read_json(f"{directory}/onlyq_ts.json")
            for i, d in enumerate(ts):
                ts[i]["Scenario"] = directory.name
            ds_ts.extend(ts)

            # tr = read_json(f"{directory}/onlyq_tr.json")
            # for i, d in enumerate(tr):
            #     tr[i]["Scenario"] = directory.name
            # dt_tr.extend(tr)
    
    ds = ds_ts
    
    db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
    
    with open(db_gt_filename, "w", encoding="utf-8") as f:
        f.write("[")
        with tqdm(total=len(ds)) as pbar:
            for d in ds:
                pbar.set_description(f"Processing {d['Input']}")
                # print("--")
                
                input = d["Input"]
                # print(f"Input: {input}")
                scenario = d["Scenario"]
                
                response = d["Response"]
                instruction_set = response["Instruction Set"]
                # print(f"Instruction Set: {type(instruction_set)}, {len(instruction_set)}")
                
                input_report = run_query_and_get_report(input, scenario, instruction_set)
                
                try:
                    # print(input_report)
                    f.write(json.dumps(input_report, ensure_ascii=False) + ",\n")
                except Exception as e:
                    logger.error(f"Error outside: {e}")
                    logger.error(f"Invoked with Input: {input}")
                    logger.error(f"Input Report: {input_report}")
                    # exit()
                    raise e
                
                # print("\n")
                pbar.update(1)
    
        # make it json array format
        # remove last comma
        f.seek(f.tell() - 2, 0)
        f.write("]")

def product_query_info(result):
    cols = result["result_columns"]
    rows = result["result_indices"]
    return list(itertools.product(cols, rows))

build_query_groundtruth()
    

Processing 지난 3일 동안 우리반 실내 온도 평균 값 알려줘.: 100%|██████████| 35/35 [00:00<00:00, 36.36it/s]                  


# Eval_query

In [5]:
def eval_query(db_gt_filename, cand_response_filename):
    db_gts = read_json(db_gt_filename)
    cand_responses = read_json(cand_response_filename)

    evaluation_reports = []

    with tqdm(total=len(cand_responses)) as pbar:
        for cand_response in cand_responses:
            pbar.set_description(f"Processing {cand_response['Input']}")
            input = cand_response["Input"]
            scenario = cand_response["Scenario"]
            
            # 관계 없는 질문들은 건너뛰자
            gt_report = [d for d in db_gts if d["Input"] == input and d["Scenario"] == scenario]
            assert len(gt_report) == 1
            gt_report = gt_report[0]
            if gt_report["Result"] == []:
                pbar.update(1)
                continue
            
            evaluation_report = defaultdict(lambda: None)
            evaluation_report["Input"] = input
            evaluation_report["Scenario"] = scenario
            
            if isinstance(cand_response["Candidate"], dict):
                cand_instruction_set = cand_response["Candidate"]["Instruction Set"]
                evaluation_report[EM.json_structure] = True
            else:
                evaluation_report[EM.json_structure] = False
                try:
                    import re
                    # get data between "Instruction Set": [ and the last]
                    cand_instruction_set = re.search(r'(?<="Instruction Set": \[)(.*)(?=\])', cand_response["Candidate"], re.DOTALL).group(0)
                    # find all {"type": ~ }, {"type": ~ }, {"type": ~ }
                    cand_instruction_set = re.findall(r'({"type".*?})', cand_instruction_set)
                    # print(list(cand_instruction_set))
                    cand_instruction_set = [eval(d) for d in cand_instruction_set]
                except Exception as e:
                    evaluation_report[EM.json_structure] = False
                    print("Failed to parse input: ", input, cand_response["Candidate"])
                    print(e)
                    evaluation_reports.append(evaluation_report)
                    pbar.update(1)
                    print(evaluation_report)
                    continue
                    
            cand_report = run_query_and_get_report(input, scenario, cand_instruction_set) 
            
            # print(f"Input: {input}")
            
            gt_results, cand_results = gt_report["Result"], cand_report["Result"]
            gt_flatten, cand_flatten = [product_query_info(r) for r in gt_results], [product_query_info(r) for r in cand_results]
            # flatten them
            gt_flatten, cand_flatten = [item for sublist in gt_flatten for item in sublist], [item for sublist in cand_flatten for item in sublist]
            
            # print(len(gt_flatten), len(cand_flatten))
            
            
            # check if all gt results are in cand results
            true_positive, false_positive, false_negative = 0, 0, 0
            for gt_data in gt_flatten:
                try:
                    cand_flatten.remove(gt_data)
                    true_positive += 1
                except ValueError as e:
                    false_negative += 1
            
            false_positive = len(cand_flatten)
            
            evaluation_report[EM.true_positive] = true_positive
            evaluation_report[EM.false_positive] = false_positive
            evaluation_report[EM.false_negative] = false_negative
            
            evaluation_reports.append(evaluation_report)
            # print(evaluation_report)
            
            pbar.update(1)

    eval_df = pd.DataFrame(evaluation_reports)
    # print(eval_df)

    eval_df['ExactMatch'] = eval_df.apply(lambda x: x[EM.false_positive] == 0 and x[EM.false_negative] == 0, axis=1).astype(int)
    # eval_df['TruePositive'] = eval_df['TruePositive'].astype(int)
    # eval_df['FalsePositive'] = eval_df['FalsePositive'].astype(int)
    # eval_df['FalseNegative'] = eval_df['FalseNegative'].astype(int)

    final_result = {}

    for col in ["JsonStructureCorrectness", "ExactMatch"]:
        # print(f"{col}: {eval_df[col].mean()}")
        final_result[col] = eval_df[col].mean()
        
    # F1 score except nans.
    truepos_sum, falsepos_sum, falseneg_sum = eval_df[EM.true_positive].sum(), eval_df[EM.false_positive].sum(), eval_df[EM.false_negative].sum()
    precision = truepos_sum / (truepos_sum + falsepos_sum)
    recall = truepos_sum / (truepos_sum + falseneg_sum)
    f1 = 2 * (precision * recall) / (precision + recall)
    # print(f"F1: {f1}")
    final_result["F1"] = f1
    final_result["Recall"] = recall
    for col in final_result:
        print(f"{col}: {final_result[col]:.2f}")
    
    return eval_df

In [13]:
db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-r128_a256_woall-checkpoint-60"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-v5_r64_a128_woall-checkpoint-72"
cand_response_filename = f"{BASE_DIR}/experiments/{cand_response_filename}.json"

eval_df = eval_query(db_gt_filename, cand_response_filename)

# print(eval_df)

Processing 이번주 우리반과 옆반 온도 변화 추이 비교해줘:   9%|▊         | 3/35 [00:00<00:00, 53.73it/s] 

Failed to parse input:  Why is our classroom so cold {"Instruction Set": [{"type": "q", "args": {"table_name": "data_t", "columns": ["raw:(SELECT name FROM idu_t WHERE id = data_t.idu_id) AS idu_name", "raw:(SELECT metadata FROM idu_t WHERE id = data_t.idu_id) AS idu_metadata", "raw:oper AS oper", "raw:roomtemp AS roomtemp", "raw:settemp AS settemp" }, "conditions": ["timestamp = '2021-08-05 14:30:00'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5')"},]}
closing parenthesis '}' does not match opening parenthesis '[' (<string>, line 1)
defaultdict(<function eval_query.<locals>.<lambda> at 0x7e60cd1cbec0>, {'Input': 'Why is our classroom so cold', 'Scenario': 'scenario3', 'JsonStructureCorrectness': False})


Processing 현재 설정온도랑 실내온도 차이 알려줘.:  11%|█▏        | 4/35 [00:00<00:03,  8.10it/s]       ERROR:db.instance:An error occurred while selecting data with
SELECT (SELECT settingtemp FROM idu_t WHERE id = data_t.idu_id) AS settemp_t, (SELECT roomtemp FROM idu_t WHERE id = data_t.idu_id) AS roomtemp_t, "id" FROM "data_t" WHERE idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5') AND timestamp = '2021-08-05 14:30:00' AND "id" IS NOT NULL
ERROR:db.instance:Error message: column "settingtemp" does not exist
LINE 1: SELECT (SELECT settingtemp FROM idu_t WHERE id = data_t.idu_...
                       ^

ERROR:__main__:Error inside: 'NoneType' object has no attribute 'columns'
ERROR:__main__:Invoked with Query: {'table_name': 'data_t', 'columns': ['raw:(SELECT settingtemp FROM idu_t WHERE id = data_t.idu_id) AS settemp_t', 'raw:(SELECT roomtemp FROM idu_t WHERE id = data_t.idu_id) AS roomtemp_t', 'id'], 'conditions': ["timestamp = '2021-08-05 14:30:00'"], 'subquery': "idu_id = (SELECT id FROM idu

JsonStructureCorrectness: 0.97
ExactMatch: 0.69
F1: 0.96
Recall: 0.98
                               Input   Scenario  JsonStructureCorrectness  \
0       Why is our classroom so cold  scenario3                     False   
1            오늘 아침과 저녁의 온도차이는 얼마나 돼?  scenario3                      True   
2               지금 옆반 온도랑 우리반 온도 알려줘  scenario3                      True   
3          이번주 우리반과 옆반 온도 변화 추이 비교해줘  scenario3                      True   
4              현재 설정온도랑 실내온도 차이 알려줘.  scenario3                      True   
5   지난주에 설정온도와 실내온도 차이가 가장 많이 났던 날은?  scenario3                      True   
6          어제 옆반 에어컨 전원 언제 꺼졌는지 알려줘.  scenario3                      True   
7                        앞반 전원 켜져있어?  scenario3                      True   
8                          어제 전원 껐어?  scenario3                      True   
9             최근에 설정온도가 가장 높았던 날 알려줘  scenario3                      True   
10      지난 3일 동안 우리반 실내 온도 평균 값 알려줘.  scenario3                      True   
11    

In [14]:
eval_df[eval_df["ExactMatch"] == 0][["Input", "Scenario", "FalsePositive", "FalseNegative"]]

,Input,Scenario,FalsePositive,FalseNegative
0,Why is our classroom so cold,scenario3,NaN,NaN
1,오늘 아침과 저녁의 온도차이는 얼마나 돼?,scenario3,5036.0,0.0
4,현재 설정온도랑 실내온도 차이 알려줘.,scenario3,0.0,3.0
10,지난 3일 동안 우리반 실내 온도 평균 값 알려줘.,scenario3,2880.0,0.0
12,오늘 아침과 저녁의 온도차이는 얼마나 돼?,scenario1,4978.0,0.0
15,현재 설정온도랑 실내온도 차이 알려줘.,scenario1,0.0,3.0
22,Why is our classroom so cold,scenario2,7115.0,7165.0
23,오늘 아침과 저녁의 온도차이는 얼마나 돼?,scenario2,5692.0,724.0
24,지금 옆반 온도랑 우리반 온도 알려줘,scenario2,4.0,4.0
26,현재 설정온도랑 실내온도 차이 알려줘.,scenario2,3.0,3.0


In [ ]:
db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-r256_a512_FI-checkpoint-57"
cand_response_filename = f"{BASE_DIR}/experiments/{cand_response_filename}.json"

eval_df = eval_query(db_gt_filename, cand_response_filename)

print(eval_df)

In [ ]:
eval_df

In [ ]:
db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-r256_a512_ISP-checkpoint-104"
cand_response_filename = f"{BASE_DIR}/experiments/{cand_response_filename}.json"

eval_df = eval_query(db_gt_filename, cand_response_filename)


In [ ]:
(eval_df)

In [9]:
db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-r128_a256_ours-checkpoint-82"
# cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-r256_a512_ours-checkpoint-138"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-v5_r256_a512_ours-checkpoint-54"
cand_response_filename = "response-sh2orc-Llama-3.1-Korean-8B-Instruct-v5_r64_a128_ours-checkpoint-60"

cand_response_filename = f"{BASE_DIR}/experiments/{cand_response_filename}.json"

eval_df = eval_query(db_gt_filename, cand_response_filename)


Processing 지난 3일 동안 우리반 실내 온도 평균 값 알려줘.: 100%|██████████| 35/35 [00:56<00:00,  1.63s/it]                  

JsonStructureCorrectness: 1.00
ExactMatch: 0.56
F1: 0.86
Recall: 0.77


In [10]:
eval_df[eval_df["ExactMatch"] == 0][["Input", "Scenario", "FalsePositive", "FalseNegative"]]

,Input,Scenario,FalsePositive,FalseNegative
0,Why is our classroom so cold,scenario3,0,7200
3,이번주 우리반과 옆반 온도 변화 추이 비교해줘,scenario3,0,20160
9,최근에 설정온도가 가장 높았던 날 알려줘,scenario3,0,10080
10,지난 3일 동안 우리반 실내 온도 평균 값 알려줘.,scenario3,2880,0
11,Why is our classroom so cold,scenario1,0,7115
14,이번주 우리반과 옆반 온도 변화 추이 비교해줘,scenario1,0,14201
21,지난 3일 동안 우리반 실내 온도 평균 값 알려줘.,scenario1,2846,2836
22,Why is our classroom so cold,scenario2,1071,7165
23,오늘 아침과 저녁의 온도차이는 얼마나 돼?,scenario2,244,724
24,지금 옆반 온도랑 우리반 온도 알려줘,scenario2,4,4
